In [23]:
import requests
from typing import List, Dict
from polling2 import poll_decorator

with open("../sec/alpha_vantage.key", "r") as file:
    av_key: str = file.read()

av_base_url: str = 'https://www.alphavantage.co/query?function='
def get_from_av(url: str) -> Dict:
    req = requests.get(url)
    print(req.json())
    req.json()

def get_fx_pair(currency_from: str, currency_to: str) -> Dict:
    url: str = f'{av_base_url}CURRENCY_EXCHANGE_RATE&from_currency={currency_from}&to_currency={currency_to}&apikey={av_key}'
    req = get_from_av(url)
    req

# @poll_decorator(step=1, timeout=15)
# def wait_until_exists(uri):
#     return requests.get(uri).status_code != 404

In [24]:
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
# url = f'https://www.alphavantage.co/query?function=CURRENCY_EXCHANGE_RATE&from_currency=USD&to_currency=JPY&apikey={av_key}'
# r = requests.get(url)
# data = r.json()

currency_pairs: List[List[str]] = [['USD', 'GBP'], ['USD', 'EUR'], ['USD', 'JPY'], ['USD', 'CAD'], ['USD', 'CHF'], ['EUR', 'GBP'], ['EUR', 'JPY'], ['EUR', 'CHF'], ['USD', 'BRL'], ['USD', 'CNY'], ['USD', 'INR']]

pairs: Dict[str,Dict] = {}
for p in currency_pairs:
    val = get_fx_pair(p[0], p[1])
    pairs[p[0] + p[1]] = val

print(pairs)

https://www.alphavantage.co/query?function=CURRENCY_EXCHANGE_RATE&from_currency=USD&to_currency=GBP&apikey=RU28ZGGQLQXHQNUP
{'Note': 'Thank you for using Alpha Vantage! Our standard API call frequency is 5 calls per minute and 500 calls per day. Please visit https://www.alphavantage.co/premium/ if you would like to target a higher API call frequency.'}
None
None
https://www.alphavantage.co/query?function=CURRENCY_EXCHANGE_RATE&from_currency=USD&to_currency=EUR&apikey=RU28ZGGQLQXHQNUP
{'Note': 'Thank you for using Alpha Vantage! Our standard API call frequency is 5 calls per minute and 500 calls per day. Please visit https://www.alphavantage.co/premium/ if you would like to target a higher API call frequency.'}
None
None
https://www.alphavantage.co/query?function=CURRENCY_EXCHANGE_RATE&from_currency=USD&to_currency=JPY&apikey=RU28ZGGQLQXHQNUP
{'Note': 'Thank you for using Alpha Vantage! Our standard API call frequency is 5 calls per minute and 500 calls per day. Please visit https://www